### keras-yolo3 패키지를 이용하여 Yolo와 tiny Yolo 기반으로 이미지와 영상 Object Detection 수행
* 다크넷에서 Pretrained된 yolo/tiny-yolo weights 모델을 다운로드
* 다운로드한 다크넷 weight 파일을 기반으로 keras-yolo3에서 사용할 수 있는 weight 파일로 변환 후 이를 이용하여 Object Detection 수행

In [ ]:
import os
import sys
import random
import math
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt

#### Local Directory 상에서 yolo package를 import 함. 
* keras-yolo3는 setup을 제공하지 않으므로 local directory상에서 바로 package를 import함. 
* 이를 위해 keras-yolo3를 system path에 추가  
* keras-yolo3 디렉토리에 있는 yolo.py에서 YOLO class를 import하여 사용. 

In [ ]:
LOCAL_PACKAGE_DIR = os.path.abspath("./keras-yolo3")
sys.path.append(LOCAL_PACKAGE_DIR)

from yolo import YOLO

In [ ]:
# YOLO 클래스는 model_path, achors_path, classes_path를 model_data 밑에 파일로 가짐. 변경을 위해서는 yolo.py 파일에서 YOLO 클래스코드를  직접 변경 필요. 

print(LOCAL_PACKAGE_DIR)
!ls /home/chulmin.kwon45/DLCV/Detection/yolo/keras-yolo3
!cat /home/chulmin.kwon45/DLCV/Detection/yolo/keras-yolo3/yolo.py

#### YOLO 객체를 사용하기 위한 모델 파일 설정 및 소스 코드 변경
* 다크넷에서 Yolo V3 Weight 모델 파일을 다운로드 받은 뒤 이를 keras-yolo3용으로 모델 파일 변경
* model_data 디렉토리 밑에 yolo_anchors.txt, coco_classes.txt 가 있는지 확인.


In [ ]:
# keras-yolo3 디렉토리 밑에서 아래 명령어로 다크넷에서 Yolov3 weight를 다운로드 받고 이를 keras-yolo3용으로 모델 파일 변경
#!wget https://pjreddie.com/media/files/yolov3.weights
#!python convert.py yolov3.cfg yolov3.weights model_data/yolo.h5
# convert.py를 수행하면 yolo anchor값이 yolo_anchors.txt 파일이 자동으로 생성됨. 
# coco label과 클래스 매핑은 0부터 매핑함.(0 => person)

In [ ]:
import sys
import argparse
from yolo import YOLO, detect_video
#keras-yolo에서 image처리를 주요 PIL로 수행. 
from PIL import Image

# YOLO 객체 생성. config는 default로 keras-yolo3 디렉토리에 있는 yolov3.cfg를 적용. 
config_dict = {}
yolo = YOLO(model_path='~/DLCV/Detection/yolo/keras-yolo3/model_data/yolo.h5',
            anchors_path='~/DLCV/Detection/yolo/keras-yolo3/model_data/yolo_anchors.txt',
            classes_path='~/DLCV/Detection/yolo/keras-yolo3/model_data/coco_classes.txt')

#### 단일 이미지 Object Detection 

In [ ]:
# 원본 이미지 보기 
img = Image.open(os.path.join('../../data/image/beatles01.jpg'))

plt.figure(figsize=(12, 12))
plt.imshow(img)

In [ ]:
# yolo.detect_image() 메소드는 PIL package를 이용하여 image 작업 수행. keras-yolo3/font 디렉토리를 상위 디렉토리로 복사 해야함.  

In [ ]:
img = Image.open(os.path.join('../../data/image/beatles01.jpg'))
detected_img = yolo.detect_image(img)

plt.figure(figsize=(12, 12))
plt.imshow(detected_img)

### Video Object Detection 수행

In [ ]:
import cv2
import time

def detect_video_yolo(model, input_path, output_path=""):
    
    start = time.time()
    cap = cv2.VideoCapture(input_path)
    
    #codec = cv2.VideoWriter_fourcc(*'DIVX')
    codec = cv2.VideoWriter_fourcc(*'XVID')
    vid_fps = cap.get(cv2.CAP_PROP_FPS)
    vid_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    vid_writer = cv2.VideoWriter(output_path, codec, vid_fps, vid_size)
    
    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print('총 Frame 갯수:', frame_cnt, '원본 영상 FPS:',vid_fps)
    index = 0
    while True:
        hasFrame, image_frame = cap.read()
        if not hasFrame:
            print('프레임이 없거나 종료 되었습니다.')
            break
        start = time.time()
        # PIL Package를 내부에서 사용하므로 cv2에서 읽은 image_frame array를 다시 PIL의 Image형태로 변환해야 함.  
        image = Image.fromarray(image_frame)
        # 아래는 인자로 입력된 yolo객체의 detect_image()로 변환한다.
        detected_image = model.detect_image(image)
        # cv2의 video writer로 출력하기 위해 다시 PIL의 Image형태를 array형태로 변환 
        result = np.asarray(detected_image)
        index +=1
        print('#### frame:{0} 이미지 처리시간:{1}'.format(index, round(time.time()-start,3)))
        
        vid_writer.write(result)
    
    vid_writer.release()
    cap.release()
    print('### Video Detect 총 수행시간:', round(time.time()-start, 5))

In [ ]:
detect_video_yolo(yolo, '../../data/video/Night_Day_Chase.mp4', '../../data/output/Night_Day_Chase_yolo_01.avi')

### tiny Yolo를 이용하여 이미지와 영상 object detection 수행. 
* tiny yolo weights파일은 https://pjreddie.com/media/files/yolov3-tiny.weights 에서 다운로드 받을 수 있음. 
* 다운로드 받은 tiny-yolo weight파일을 keras-yolo3에서 사용할 수 있게 Convert 수행 후 YOLO객체에서 로딩하여 사용 

In [ ]:
# wget https://pjreddie.com/media/files/yolov3-tiny.weights
#!python convert.py yolov3-tiny.cfg ./model_data/yolov3-tiny.weights model_data/yolo-tiny.h5

#### tiny yolo weight 파일과 anchor 파일, coco 클래스 파일을 YOLO 객체 생성 시 인자로 입력

In [ ]:
tiny_yolo = YOLO(model_path='~/DLCV/Detection/yolo/keras-yolo3/model_data/yolo-tiny.h5',
            anchors_path='~/DLCV/Detection/yolo/keras-yolo3/model_data/tiny_yolo_anchors.txt',
            classes_path='~/DLCV/Detection/yolo/keras-yolo3/model_data/coco_classes.txt')

#### 단일 이미지 Object Detection

In [ ]:
img = Image.open(os.path.join('../../data/image/beatles01.jpg'))
detected_img = tiny_yolo.detect_image(img)

plt.figure(figsize=(12, 12))
plt.imshow(detected_img)

#### Video Object Detection

In [ ]:
detect_video_yolo(tiny_yolo, '../../data/video/secretlife_pet.mp4', '../../data/output/secretlife_pet_yolo01.avi')